# Загрузка данных анкет и проведение МАИ, анализ полученных результатов
Данные: заполненные анкеты экспертов (таблицы Excel) 

In [1]:
import re
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from random import randint

In [2]:
df = pd.read_excel(r'C:\Users\user\предпосылки\Экс12.xlsx')
df.head()

,Тип нарушения,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала
0,Документация,1.000000,2.000000,3.0,1.0,2.0,4.0,3
1,Экспертиза промышленной безопасности,0.500000,1.000000,5.0,7.0,3.0,5.0,3
2,Готовность к ликвидации последствий аварий и и...,0.333333,0.200000,1.0,5.0,1.0,6.0,3
3,Оформление информационными знаками и знаками б...,1.000000,0.142857,0.2,1.0,2.0,1.0,3
4,Содержание объектов и оборудования,0.500000,0.333333,1.0,0.5,1.0,7.0,1


In [3]:
print('Число строк и столбцов в наборе данных:\n', df.shape)

Число строк и столбцов в наборе данных:
 (7, 8)


In [4]:
# типы столбцов 
df.dtypes

Тип нарушения                                                 object
Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала         int64
dtype: object

In [5]:
#удаление столбца типа object для дальнейшей работы с фреймом 
df.pop('Тип нарушения') 
print(df)

   Документация   Экспертиза промышленной безопасности  \
0       1.000000                              2.000000   
1       0.500000                              1.000000   
2       0.333333                              0.200000   
3       1.000000                              0.142857   
4       0.500000                              0.333333   
5       0.250000                              0.200000   
6       0.333333                              0.333333   

   Готовность к ликвидации последствий аварий и инцидентов  \
0                                           3.000000         
1                                           5.000000         
2                                           1.000000         
3                                           0.200000         
4                                           1.000000         
5                                           0.166667         
6                                           0.333333         

   Оформление информационными знаками 

In [6]:
# типы столбцов 
df.dtypes

Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала         int64
dtype: object

In [7]:
# также меням тип данных Документации
df['Документация '] = df['Документация '].astype(float)

In [8]:
df.loc[:, df.columns[df.dtypes == 'float64']].head(7)

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ)
0,1.000000,2.000000,3.000000,1.000000,2.000000,4.0
1,0.500000,1.000000,5.000000,7.000000,3.000000,5.0
2,0.333333,0.200000,1.000000,5.000000,1.000000,6.0
3,1.000000,0.142857,0.200000,1.000000,2.000000,1.0
4,0.500000,0.333333,1.000000,0.500000,1.000000,7.0
5,0.250000,0.200000,0.166667,1.000000,0.142857,1.0
6,0.333333,0.333333,0.333333,0.333333,1.000000,0.5


In [9]:
DF_1=df[:].copy()

In [10]:
#Найдем сумму всех столбцов, для того чтобы в дальнейшем пересчитать полученные значения
df.sum ()

Документация                                                  3.916667
Экспертиза промышленной безопасности                          4.209524
Готовность к ликвидации последствий аварий и инцидентов      10.700000
Оформление информационными знаками и знаками безопасности    15.833333
Содержание объектов и оборудования                           10.142857
Охранные зоны (ОЗ)                                           24.500000
Организация рабочего процесса и безопасности персонала       16.000000
dtype: float64

In [11]:
# Считаем значения для нашей матрицы, путем деления каждого значения в столбце на сумму столбца
df['new_Документация'] = df['Документация '].astype(float).div(df['Документация '].sum())
df['new_Экспертиза'] = df['Экспертиза промышленной безопасности'].astype(float).div(df['Экспертиза промышленной безопасности'].sum())
df['new_Готовность'] = df['Готовность к ликвидации последствий аварий и инцидентов'].astype(float).div(df['Готовность к ликвидации последствий аварий и инцидентов'].sum())
df['new_Оформление'] = df['Оформление информационными знаками и знаками безопасности'].astype(float).div(df['Оформление информационными знаками и знаками безопасности'].sum())
df['new_Содержание'] = df['Содержание объектов и оборудования '].astype(float).div(df['Содержание объектов и оборудования '].sum())
df['new_ОЗ'] = df['Охранные зоны (ОЗ)'].astype(float).div(df['Охранные зоны (ОЗ)'].sum())
df['new_Организация'] = df['Организация рабочего процесса и безопасности персонала'].astype(float).div(df['Организация рабочего процесса и безопасности персонала'].sum())

In [12]:
df.head()

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,1.000000,2.000000,3.0,1.0,2.0,4.0,3,0.255319,0.475113,0.280374,0.063158,0.197183,0.163265,0.1875
1,0.500000,1.000000,5.0,7.0,3.0,5.0,3,0.127660,0.237557,0.467290,0.442105,0.295775,0.204082,0.1875
2,0.333333,0.200000,1.0,5.0,1.0,6.0,3,0.085106,0.047511,0.093458,0.315789,0.098592,0.244898,0.1875
3,1.000000,0.142857,0.2,1.0,2.0,1.0,3,0.255319,0.033937,0.018692,0.063158,0.197183,0.040816,0.1875
4,0.500000,0.333333,1.0,0.5,1.0,7.0,1,0.127660,0.079186,0.093458,0.031579,0.098592,0.285714,0.0625


In [13]:
# Получили новую таблицу теперь отделим значения нашей матрицы от основной таблицы
df_new = df[['new_Документация', 'new_Экспертиза', 'new_Готовность', 'new_Оформление', 'new_Содержание', 'new_ОЗ', 'new_Организация']]
df_new

,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,0.255319,0.475113,0.280374,0.063158,0.197183,0.163265,0.1875
1,0.127660,0.237557,0.467290,0.442105,0.295775,0.204082,0.1875
2,0.085106,0.047511,0.093458,0.315789,0.098592,0.244898,0.1875
3,0.255319,0.033937,0.018692,0.063158,0.197183,0.040816,0.1875
4,0.127660,0.079186,0.093458,0.031579,0.098592,0.285714,0.0625
5,0.063830,0.047511,0.015576,0.063158,0.014085,0.040816,0.1250
6,0.085106,0.079186,0.031153,0.021053,0.098592,0.020408,0.0625


In [14]:
#Вычислим средние значение по строке и запишем полученный результот в mean
df_new['mean'] = df_new.mean(axis=1)
df_new.head()

C:\Users\user\anaconda3\envs\envpy3.5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация,mean
0,0.255319,0.475113,0.280374,0.063158,0.197183,0.163265,0.1875,0.231702
1,0.127660,0.237557,0.467290,0.442105,0.295775,0.204082,0.1875,0.280281
2,0.085106,0.047511,0.093458,0.315789,0.098592,0.244898,0.1875,0.153265
3,0.255319,0.033937,0.018692,0.063158,0.197183,0.040816,0.1875,0.113801
4,0.127660,0.079186,0.093458,0.031579,0.098592,0.285714,0.0625,0.111241


In [15]:
#проверка, среднее значение в сумме должно быть равно 1
df_new['mean'].sum()

1.0

In [16]:
# Далее проведем проверку на согласованность, 
#для этого создадим из нашей полученной выше таблицы матрицы для дальнейшего их перемножения
df_new8 =df_new['mean']

# удаляем из нашей таблицы значениея столбцы mean
#df_new.pop('mean')

df_new1 =DF_1.loc[0]

df_new2 =DF_1.loc[1]

df_new3 =DF_1.loc[2]

df_new4 =DF_1.loc[3]

df_new5 =DF_1.loc[4]

df_new6 =DF_1.loc[5]

df_new7 =DF_1.loc[6]

In [17]:
#Проверка на согласованность
res1 = np.dot(df_new1,df_new8)
print(res1)
res2 = np.dot(df_new2,df_new8)
print(res2)
res3 = np.dot(df_new3,df_new8)
print(res3)
res4 = np.dot(df_new4,df_new8)
print(res4)
res5 = np.dot(df_new5,df_new8)
print(res5)
res6 = np.dot(df_new6,df_new8)
print(res6)
res7 = np.dot(df_new7,df_new8)
print(res7)

1.970326672633064
2.72762351129872
1.4544920759877438
0.8621016532809513
0.9575171554884594
0.4357852076116347
0.45420750014994477


In [18]:
sum_new = res1 + res2 + res3 + res4 + res5 + res6 + res7
print(sum_new)

8.86205377645052


In [19]:
# Вычислем значения CI, RI, CR
CI = (sum_new - 7)/(7-1)
print(CI)
RI = (1.98*(7-2))/7
print(RI)
CR = CI/RI
print(CR)

0.31034229607508657
1.4142857142857144
0.21943394671975816


In [20]:
# Получилось, что значение CR=0.22, что большее 0,1 значит матрицу нельзя считать согласованной
# Проделаем аналогичные действия с другими экспертами